# Extract data from nifty image files to bmp

In [1]:
from PIL import Image
import glob
import shutil
import os
import numpy as np
import nibabel as nib
import torchvision.transforms
import matplotlib.pyplot as plt
%matplotlib widget

In [2]:
root_dir = "G:/MachineLearning/training-batch-1/testing/"
image_files = sorted(glob.glob(root_dir + 'volume*.nii'))
mask_files = sorted(glob.glob(root_dir + 'segmentation*.nii'))

In [3]:
out_dir = "G:/MachineLearning/liver-database/validation/"

window_min = -250
window_max = 250

downsample = torchvision.transforms.Compose(
        [torchvision.transforms.ToPILImage(),
         torchvision.transforms.Resize(256, interpolation=torchvision.transforms.InterpolationMode.BILINEAR, antialias=True),
         torchvision.transforms.ToTensor()]
    )
downsample_mask = torchvision.transforms.Compose(
    [torchvision.transforms.ToPILImage(),
     torchvision.transforms.Resize(256, interpolation=torchvision.transforms.InterpolationMode.NEAREST),
     torchvision.transforms.ToTensor()]
)

num_img = len(image_files)
for i in range(num_img):
    image = nib.load(image_files[i])
    data = image.get_fdata()
    num_slices = data.shape[2] # 512 x 512 x num_slices
    for j in range(num_slices):
        slc = data[:, :, j]
        slc[slc < window_min] = window_min
        slc[slc > window_max] = window_max
        slc = (slc - window_min) / (window_max - window_min)
        # slc = downsample(slc.astype(np.float32)).numpy()
        slc.astype(np.float16).tofile(out_dir + "images/" + os.path.splitext(os.path.basename(image_files[i]))[0] + ("-%04d.ct" % j))

        
num_img = len(mask_files)
for i in range(num_img):
    image = nib.load(mask_files[i])
    data = image.get_fdata()
    num_slices = data.shape[2] # 512 x 512 x num_slices
    for j in range(num_slices):
        slc = data[:, :, j]
        # slc = downsample_mask(slc.astype(np.float32)).numpy()
        slc.astype(np.float16).tofile(out_dir + "masks/" + os.path.splitext(os.path.basename(mask_files[i]))[0] + ("-%04d.ct" % j))